In [ ]:
import sagemaker
from sagemaker.sklearn.model import SKLearnModel

# Configuración de SageMaker
role = "arn:aws:iam::193862962620:role/SageMakerExecutionRole"  # Usa tu ARN de IAM
bucket = "ml-tigo-data"
model_path = f"s3://{bucket}/modelos/best_model.tar.gz"

# Usar la imagen de Scikit-Learn para cargar el modelo
model = SKLearnModel(
    model_data="s3://ml-tigo-data/modelos/best_model.tar.gz",  # 🔥 Confirma la ruta del modelo en S3
    role="arn:aws:iam::193862962620:role/SageMakerExecutionRole",
    entry_point="inference.py",
    source_dir=".",
    dependencies=["requirements.txt"],
    framework_version="1.2-1"
)

# Desplegar el endpoint en SageMaker
predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.2xlarge"
)

/opt/conda/lib/python3.12/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


[05/05/25 10:59:43] INFO     Creating model with name:                                              ]8;id=301069;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=666409;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py#4094\4094]8;;\
                             sagemaker-scikit-learn-2025-05-05-10-59-43-874                                        

[05/05/25 10:59:44] INFO     Creating endpoint-config with name                                     ]8;id=915605;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=578696;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py#5937\5937]8;;\
                             sagemaker-scikit-learn-2025-05-05-10-59-44-600                                        

                    INFO     Creating endpoint with name                                            ]8;id=65702;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=251395;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py#4759\4759]8;;\
                             sagemaker-scikit-learn-2025-05-05-10-59-44-600                                        

-

In [ ]:
import sagemaker

sm_client = sagemaker.Session().sagemaker_client

response = sm_client.describe_endpoint(EndpointName=predictor.endpoint_name)
print(response["EndpointStatus"])  # Debe devolver "InService"

In [3]:
import boto3
import json
import pandas as pd
import os

from transform_functions import fun_final_base, NonNullValues, new_vars_trend, new_vars_trend, encoder_scaler_loading, const_vars_serv, transf_PCA_POL

# Cliente de SageMaker Runtime
sagemaker_runtime = boto3.client("sagemaker-runtime")

# Nombre del endpoint
endpoint_name = predictor.endpoint_name  # Ajusta según tu configuración

# Leer el CSV y convertirlo en JSON
df = pd.read_csv("df_muestra.csv")  # Reemplaza con la ruta correcta
payload = df.to_json(orient="records")  # Convierte los datos en formato JSON

# Enviar la solicitud al modelo
response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=payload
)

# Mostrar resultado
result = json.loads(response["Body"].read().decode("utf-8"))
print("🔮 Predicciones:", result)

✅ Ahora transform_functions está en: /home/sagemaker-user
📌 Este script está en: /home/sagemaker-user


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:19                                                                                   │
│                                                                                                  │
│   16 payload = df.to_json(orient="records")  # Convierte los datos en formato JSON               │
│   17                                                                                             │
│   18 # Enviar la solicitud al modelo                                                             │
│ ❱ 19 response = sagemaker_runtime.invoke_endpoint(                                               │
│   20 │   EndpointName=endpoint_name,                                                             │
│   21 │   ContentType="application/json",                                                         │
│   22 │   Body=payload                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/client.py:569 in _api_call                      │
│                                                                                                  │
│    566 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    567 │   │   │   │   )                                                                         │
│    568 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  569 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    570 │   │                                                                                     │
│    571 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    572                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/client.py:1023 in _make_api_call                │
│                                                                                                  │
│   1020 │   │   │   │   "Code"                                                                    │
│   1021 │   │   │   )                                                                             │
│   1022 │   │   │   error_class = self.exceptions.from_code(error_code)                           │
│ ❱ 1023 │   │   │   raise error_class(parsed_response, operation_name)                            │
│   1024 │   │   else:                                                                             │
│   1025 │   │   │   return parsed_response                                                        │
│   1026                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) 
from primary with message "<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is 
overloaded or there is an error in the application.</p>
". See 
https://us-east-2.console.aws.amazon.com/cloudwatch/home?region=us-east-2#logEventViewer:group=/aws/sagemaker/Endpo
ints/sagemaker-scikit-learn-2025-05-05-10-46-35-419 in account 193862962620 for more information.